In [32]:
import os
def get_comment(code_str):
    code_template = """你现在是一个程序员助手. 程序员在写了代码之后忘记给代码注释. 你的任务是给程序员的代码进行合理恰当的中文注释。让用户能够快速的理解相关的代码。只输出注释后的代码。不输出其它任何多余的内容.
    Q:
    class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)
    A:
    ```python
    class StreamHandler(BaseCallbackHandler):
    \"\"\"
    StreamHandler 类用于处理从语言模型（LLM）接收的新标记并更新一个容器的显示内容。

    这个类继承自 BaseCallbackHandler，是一个回调处理器，专门用于处理和响应来自语言模型的新生成的标记。

    Attributes:
        container: 容器对象，用于在其中显示文本。
        text: 初始文本，用于在容器中开始显示。默认为空字符串。

    Methods:
        on_llm_new_token(token: str, **kwargs): 当从语言模型接收到新标记时调用。将新标记附加到现有文本并更新容器的显示。
    \"\"\"

    def __init__(self, container, initial_text=""):
        \"\"\"
        初始化 StreamHandler 实例。

        Args:
            container: 容器对象，用于显示来自语言模型的文本。
            initial_text: 可选；初始文本字符串，用于在容器中开始显示。默认为空字符串。
        \"\"\"
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        \"\"\"
        当从语言模型接收到新标记时被调用的方法。

        这个方法将接收到的标记附加到类实例的文本属性上，并更新容器中的显示内容。

        Args:
            token: 从语言模型接收到的新标记字符串。
            **kwargs: 可选参数，用于额外的功能或处理。
        \"\"\"
        self.text += token
        self.container.markdown(self.text)
        ```
    Q:
    """
    
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", code_template),
            ("human", "{input}")
        ]
    )
    base = os.getenv('OPENAI_API_BASE')
    key = os.getenv('OPENAI_API_KEY')
    llm = ChatOpenAI(
        openai_api_base=base,
        openai_api_key=key,
        model_name="gpt-3.5-turbo",
        temperature=0,
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()]
    )
    # llm = Ollama(model="yi:34b-chat")
    chain = prompt | llm
    response = chain.invoke({"input": code_str})
    # print(f"Response is {response.content}")
    # rt = extract_code(response.content)
    # return rt
    if not response.content:
        return ""
    return response.content

In [11]:
import os
import shutil
from pprint import pprint 
import warnings
import re
warnings.filterwarnings("ignore")
from langchain.document_loaders import TextLoader
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import TextSplitter
from langchain.text_splitter import Language
from langchain.schema.output_parser import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.llms import Ollama
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from operator import itemgetter


In [33]:
def get_splitter(file):
    python_splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.PYTHON, chunk_size=2000, chunk_overlap=0
    )

    java_splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.JAVA, chunk_size=2000, chunk_overlap=0
    )
    cpp_splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.CPP, chunk_size=2000, chunk_overlap=0
    )
    Javascript_splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.JS, chunk_size=2000, chunk_overlap=0
    )
    match file.split('.'):
        case [_, 'py']:
            print("Python splitter")
            return python_splitter
        case [_, 'java']:
            print("Java splitter")
            return java_splitter
        case [_, 'cpp']:
            print("C++ splitter")
            return cpp_splitter
        case [_, 'js']:
            print("Javascript splitter")
            return Javascript_splitter
        case _:
            return None

In [8]:
splitter = get_splitter(file_path)
splitter

Java splitter


In [9]:
file_path = os.path.join("data", "Simple-Java-Calculator", "src", "simplejavacalculator", "Calculator.java")
loader = TextLoader(file_path)
documents = loader.load()
texts = splitter.split_documents(documents)
len(texts)

3

In [ ]:
for  t in texts:
    print("--------------------------------------------------")
    print(t.page_content)

In [ ]:
s_lst = [get_comment(t.page_content) for t in texts]

In [34]:
rt = []
for t in texts:
    one = get_comment(t.page_content)
    print(one)
    rt.append(one)
rt

```java
package simplejavacalculator;

import static java.lang.Double.NaN;
import static java.lang.Math.log;
import static java.lang.Math.log10;
import static java.lang.Math.pow;
```
这段代码是导入了一些静态方法和常量。其中包括：
- `NaN`：表示不是一个数字的常量。
- `log`：计算自然对数的静态方法。
- `log10`：计算以10为底的对数的静态方法。
- `pow`：计算指定次幂的静态方法。```java
package simplejavacalculator;

import static java.lang.Double.NaN;
import static java.lang.Math.log;
import static java.lang.Math.log10;
import static java.lang.Math.pow;
```
这段代码是导入了一些静态方法和常量。其中包括：
- `NaN`：表示不是一个数字的常量。
- `log`：计算自然对数的静态方法。
- `log10`：计算以10为底的对数的静态方法。
- `pow`：计算指定次幂的静态方法。
```java
public class Calculator {
    /**
     * 双操作符模式枚举类，包含常见的双操作符模式。
     */
    public enum BiOperatorModes {
        normal, add, minus, multiply, divide , xpowerofy 
    }

    /**
     * 单操作符模式枚举类，包含常见的单操作符模式。
     */
    public enum MonoOperatorModes {
        square, squareRoot, oneDividedBy, cos, sin, tan, log, rate, abs, ln,
    }

    private Double num1, num2;
    private BiOperatorModes mod

['```java\npackage simplejavacalculator;\n\nimport static java.lang.Double.NaN;\nimport static java.lang.Math.log;\nimport static java.lang.Math.log10;\nimport static java.lang.Math.pow;\n```\n这段代码是导入了一些静态方法和常量。其中包括：\n- `NaN`：表示不是一个数字的常量。\n- `log`：计算自然对数的静态方法。\n- `log10`：计算以10为底的对数的静态方法。\n- `pow`：计算指定次幂的静态方法。',
 '```java\npublic class Calculator {\n    /**\n     * 双操作符模式枚举类，包含常见的双操作符模式。\n     */\n    public enum BiOperatorModes {\n        normal, add, minus, multiply, divide , xpowerofy \n    }\n\n    /**\n     * 单操作符模式枚举类，包含常见的单操作符模式。\n     */\n    public enum MonoOperatorModes {\n        square, squareRoot, oneDividedBy, cos, sin, tan, log, rate, abs, ln,\n    }\n\n    private Double num1, num2;\n    private BiOperatorModes mode = BiOperatorModes.normal;\n\n    /**\n     * 根据当前的操作符模式计算双操作数的结果。\n     *\n     * @return 双操作数的计算结果。\n     */\n    private Double calculateBiImpl() {\n        if (mode == BiOperatorModes.normal) {\n            return num2;\n        }\n        if (mode == BiOp

In [29]:
len(texts[1].page_content)

1640

In [22]:
rt = get_comment(texts[0].page_content)
rt

'```java\npackage simplejavacalculator;\n\nimport static java.lang.Double.NaN;\nimport static java.lang.Math.log;\nimport static java.lang.Math.log10;\nimport static java.lang.Math.pow;\n```\n这段代码是导入了一些静态方法和常量。其中包括：\n- `NaN`：表示不是一个数字的常量。\n- `log`：计算自然对数的静态方法。\n- `log10`：计算以10为底的对数的静态方法。\n- `pow`：计算指定次幂的静态方法。'

In [ ]:
rt = "-----\n".join(s_lst)
print(rt)

In [15]:
print(s_lst[-1])

```java
public Double calculateMono(MonoOperatorModes newMode, Double num) {
    // 根据不同的操作模式进行计算并返回结果

    if (newMode == MonoOperatorModes.square) {
        // 如果操作模式是平方，则返回数字的平方
        return num * num;
    }
    if (newMode == MonoOperatorModes.squareRoot) {
        // 如果操作模式是平方根，则返回数字的平方根
        return Math.sqrt(num);
    }
    if (newMode == MonoOperatorModes.oneDividedBy) {
        // 如果操作模式是倒数，则返回数字的倒数
        return 1 / num;
    }
    if (newMode == MonoOperatorModes.cos) {
        // 如果操作模式是余弦，则返回数字的余弦值
        return Math.cos(Math.toRadians(num));
    }
    if (newMode == MonoOperatorModes.sin) {
        // 如果操作模式是正弦，则返回数字的正弦值
        return Math.sin(Math.toRadians(num));
    }
    if (newMode == MonoOperatorModes.tan) {
        // 如果操作模式是正切，则返回数字的正切值
        if (num == 0 || num % 180 == 0) {
            return 0.0;
        }
        if (num % 90 == 0 && num % 180 != 0) {
            return NaN;
        }

        return Math.tan(Math.toRadians(num));
    }
    if (newMode

In [85]:
type(s_lst[0])

str

In [54]:
s_lst[-1].count("cpp")

1

In [16]:
s_lst[0]

'```java\npackage simplejavacalculator;\n\nimport static java.lang.Double.NaN;\nimport static java.lang.Math.log;\nimport static java.lang.Math.log10;\nimport static java.lang.Math.pow;\n```\n这段代码是导入了一些静态方法和常量。其中包括：\n- `NaN`：表示不是一个数字的常量。\n- `log`：计算自然对数的静态方法。\n- `log10`：计算以10为底的对数的静态方法。\n- `pow`：计算指定次幂的静态方法。'

In [17]:

from main import get_language
language = get_language(file_path)
def extract_code(language, text):
    language_patterns = {
    'js': r'```(js|javascript)\n(.*?)```',
    'python': r'```python\n(.*?)```',
    'java': r'```java\n(.*?)```',
    'cpp': r'```cpp\n(.*?)```',
}
    pattern = r'```java\n(.*?)```' 
    # pattern = language_patterns.get(language.value, r'```(\w+)\n(.*?)```')
    print(f"the pattern is {pattern}")
    matches = re.findall(pattern, text, re.DOTALL)
    if not matches:
        return ""
    return "\n".join(matches) 
    

In [139]:
print(language.value)

java


In [18]:
text = s_lst[0]
print(f"The str is {text}")
rt = extract_code(language, text)
print(f"result is {rt}")
# print("\n".join([r for r in rt]))

The str is ```java
package simplejavacalculator;

import static java.lang.Double.NaN;
import static java.lang.Math.log;
import static java.lang.Math.log10;
import static java.lang.Math.pow;
```
这段代码是导入了一些静态方法和常量。其中包括：
- `NaN`：表示不是一个数字的常量。
- `log`：计算自然对数的静态方法。
- `log10`：计算以10为底的对数的静态方法。
- `pow`：计算指定次幂的静态方法。
the pattern is ```java\n(.*?)```
result is package simplejavacalculator;

import static java.lang.Double.NaN;
import static java.lang.Math.log;
import static java.lang.Math.log10;
import static java.lang.Math.pow;



In [7]:
from enum import Enum
class MyLanguage(Enum):
    PYTHON = "python"
    JAVA = "java"
    CPP = "cpp"
    JS = "js"

